<a href="https://colab.research.google.com/github/Ecc535/VLLM_Mental_Health_Prediction/blob/main/vllm_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://huggingface.co/nvidia/Llama-3.1-Nemotron-Nano-4B-v1.1

Cloning into 'Llama-3.1-Nemotron-Nano-4B-v1.1'...
remote: Enumerating objects: 85, done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 85 (from 1)
Receiving objects: 100% (85/85), 76.07 KiB | 25.36 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Filtering content: 100% (3/3), 4.42 GiB | 73.34 MiB/s, done.
Encountered 2 file(s) that may not have been copied correctly on Windows:
	model-00002-of-00002.safetensors
	model-00001-of-00002.safetensors

See: `git lfs help smudge` for more details.


In [ ]:
!pip install -U vllm
!pip uninstall -y protobuf
!pip install protobuf==4.25.3

In [ ]:
# !nohup python -m vllm.entrypoints.openai.api_server \
#   --model Llama-3.1-Nemotron-Nano-4B-v1.1 \
#   --trust-remote-code \
#   --seed 1 \
#   --host 0.0.0.0 \
#   --port 5000 \
#   --served-model-name "Llama-Nemotron-Nano-4B" \
#   --tensor-parallel-size 1 \
#   --max-model-len 50_000 \
#   --gpu-memory-utilization 0.95 \
#   --enforce-eager \
#   --enable-auto-tool-choice \
#   --tool-parser-plugin "Llama-3.1-Nemotron-Nano-4B-v1.1/llama_nemotron_nano_toolcall_parser.py" \
#   --tool-call-parser "llama_nemotron_json" \
#   --chat-template "Llama-3.1-Nemotron-Nano-4B-v1.1/llama_nemotron_nano_generic_tool_calling.jinja" \
#   > vllm.log &

nohup: redirecting stderr to stdout


In [13]:
!nohup python -m vllm.entrypoints.openai.api_server \
  --model Llama-3.1-Nemotron-Nano-4B-v1.1 \
  --trust-remote-code \
  --host 0.0.0.0 \
  --port 5000 \
  --served-model-name Llama-Nemotron-Nano-4B \
  --tensor-parallel-size 1 \
  --max-model-len 8192 \
  --gpu-memory-utilization 0.85 \
  --enable-auto-tool-choice \
  --tool-call-parser llama3_json \
  > vllm.log 2>&1 &

**Verify instance is spinning up**

In [14]:
!while ! grep -q "Application startup complete" vllm.log; do tail -n 1 vllm.log; sleep 5; done

To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(APIServer pid=17999) INFO 02-27 20:00:56 [vllm.py:689] Asynchronous scheduling is enabled.
W0000 00:00:1772222462.308221   18138 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772222462.308221   18138 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(EngineCore_DP0 pid=18138) INFO 02-27 20:01:12 [core.py:97] Initializing a V1 LLM engine (v0.16.0) with config: model='Llama-3.1-Nemotron-Nano-4B-v1.1', speculative_config=None, tokenizer='Llama-3.1-Nemotron-Nano-4B-v1.1', skip_tokenizer_init=False, tokenizer_mode=aut

**Set up OpenAI Client**

In [4]:
from openai import OpenAI

client = OpenAI(
    base_url="http://0.0.0.0:5000/v1",
    api_key="dummy"
)

**For binary output classification task, I use reasoning off mode.**

In [5]:
import pandas as pd

In [6]:
FEATURE_GROUPS = {
    "Rhythm": [
        "steps_48phi", "steps_8phi", "steps_psd_auc_64h",
        "steps_12p_reject", "steps_128p_reject", "steps_IV"
    ],
    "Heart Rate Variability": [
        "hrstd_max", "hrstd_std", "hrmax_std", "hrmean_max",
        "hrmax_mean", "hrentropy_std", "RestingHeartRate_std", "hrmax_max"
    ],
    "Shift Pattern": [
        "shift_hours_median", "shift_type_shannon_entropy",
        "shift_hours_shannon_entropy"
    ],
    "Inactivity at Daytime": [
        "duration_entropy_non_step_std"
    ],
    "Morning Energy": [
        "happiness_morning_std", "happiness_morning_shannon_entropy",
        "sleepiness_daytime_shannon_entropy", "energy_morning_std",
        "alterness_morning_std", "happiness_morning_median"
    ],
    "Evening Energy": [
        "energy_evening_median", "relax_evening_median",
        "health_evening_std", "energy_evening_std",
        "health_evening_median", "happiness_evening_median",
        "happiness_evening_std"
    ],
    "Caffeine Intake": [
        "caffeine_cups_shannon_entropy",
        "awake_type_shannon_entropy",
        "awake_type_std",
        "awake_type_median"
    ],
    "Sleep": [
        "sleep_efficiency_median",
        "sleep_duration_max",
        "nap_duration_fitbit_max"
    ]
}

In [7]:
data = pd.read_json("/content/all_participants_features.json")
data.head()

,1002,1102,1103,1105,1108,1115,1150,1151,1153,1154,...,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316
Caffeine Intake,"{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.5, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 1.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 1.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...",...,"{'awake_type_median': 1.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon...","{'awake_type_median': 0.0, 'awake_type_shannon..."
Evening Energy,"{'energy_evening_median': 24.0, 'energy_evenin...","{'energy_evening_median': 41.0, 'energy_evenin...","{'energy_evening_median': 68.0, 'energy_evenin...","{'energy_evening_median': 80.0, 'energy_evenin...","{'energy_evening_median': 28.0, 'energy_evenin...","{'energy_evening_median': 70.5, 'energy_evenin...","{'energy_evening_median': 30.0, 'energy_evenin...","{'energy_evening_median': 35.0, 'energy_evenin...","{'energy_evening_median': 43.0, 'energy_evenin...","{'energy_evening_median': 31.0, 'energy_evenin...",...,"{'energy_evening_median': 16.0, 'energy_evenin...","{'energy_evening_median': 36.0, 'energy_evenin...","{'energy_evening_median': 0.0, 'energy_evening...","{'energy_evening_median': 30.0, 'energy_evenin...","{'energy_evening_median': 10.0, 'energy_evenin...","{'energy_evening_median': 12.5, 'energy_evenin...","{'energy_evening_median': 0.0, 'energy_evening...","{'energy_evening_median': 0.0, 'energy_evening...","{'energy_evening_median': 0.0, 'energy_evening...","{'energy_evening_median': 0.0, 'energy_evening..."
Heart Rate Variability,"{'RestingHeartRate_std': 3.201355533001174, 'h...","{'RestingHeartRate_std': 2.171339382665621, 'h...","{'RestingHeartRate_std': 1.9563385338022201, '...","{'RestingHeartRate_std': 1.687253803322248, 'h...","{'RestingHeartRate_std': 2.102938060155001, 'h...","{'RestingHeartRate_std': 1.6523191974188092, '...","{'RestingHeartRate_std': 2.317587564950268, 'h...","{'RestingHeartRate_std': 1.323591303761829, 'h...","{'RestingHeartRate_std': 2.081526510338128, 'h...","{'RestingHeartRate_std': 1.974309473688939, 'h...",...,"{'RestingHeartRate_std': 2.6592428585304493, '...","{'RestingHeartRate_std': 4.311667448966372, 'h...","{'RestingHeartRate_std': 3.649530529229613, 'h...","{'RestingHeartRate_std': 3.228659539846594, 'h...","{'RestingHeartRate_std': 2.304064261519655, 'h...","{'RestingHeartRate_std': 1.86677488863775, 'hr...","{'RestingHeartRate_std': 2.960453633193011, 'h...","{'RestingHeartRate_std': 2.97209241668783, 'hr...","{'RestingHeartRate_std': 3.001574036011307, 'h...","{'RestingHeartRate_std': 3.299605273056646, 'h..."
Inactivity at Daytime,{'duration_entropy_non_step_std': 0.7841672614...,{'duration_entropy_non_step_std': 0.2422038221...,{'duration_entropy_non_step_std': 0.2239061320...,{'duration_entropy_non_step_std': 0.2581546889...,{'duration_entropy_non_step_std': 0.3309113415...,{'duration_entropy_non_step_std': 0.3537110787...,{'duration_entropy_non_step_std': 0.3038594506...,{'duration_entropy_non_step_std': 0.2673526136...,{'duration_entropy_non_step_std': 0.2681467520...,{'duration_entropy_non_step_std': 0.3458715151...,...,{'duration_entropy_non_step_std': 0.3844192995...,{'duration_entropy_non_step_std': 0.5441071863...,{'duration_entropy_non_step_std': 0.3027201647...,{'duration_entropy_non_step_std':

In [8]:
detailed_thinking = "off"

def build_grouped_data(row):
    grouped = {}

    for group, columns in FEATURE_GROUPS.items():
        grouped[group] = {
            col: row[col]
            for col in columns
            if col in row and not pd.isna(row[col])
        }

    return grouped

In [9]:
def build_prompt(grouped_dict):
    formatted_sections = []

    for group, features in grouped_dict.items():
        section = f"## {group}\n"
        for k, v in features.items():
            section += f"{k}: {v}\n"
        formatted_sections.append(section)

    data_block = "\n".join(formatted_sections)

    prompt = f"""
You are a behavioral health data analyst.

Below is structured wearable and survey-derived feature data grouped by domain.

Write a concise professional summary describing:
- Circadian rhythm stability
- Cardiovascular variability
- Work/shift regularity
- Daytime inactivity
- Morning and evening energy patterns
- Sleep quality

Interpret patterns at a high level.
Do NOT list features individually.
Do NOT invent values.

Data:
{data_block}

Summary:
"""
    return prompt.strip()

In [59]:
def build_classification_prompt(summary):
    return f"""
You are a clinical burnout assessment assistant.

- Emotional Exhaustion: feeling emotionally drained, fatigued, or depleted.
- Depersonalization: detached, cynical, or impersonal attitudes toward others or work.
- Reduced Personal Accomplishment: feelings of incompetence or lack of achievement at work.

Based ONLY on the behavioral health summary provided,
- High
- Low

Rules:
- Output EXACTLY one word: High or Low
- Do not explain your reasoning
- Do not output anything else

Summary:
{summary}

Burnout Risk:
""".strip()

**Test single record**

In [ ]:
response = client.chat.completions.create(
    model="Llama-Nemotron-Nano-4B",
    messages=[
        {"role": "user", "content": build_prompt(grouped_data)}
    ],
    # reasoning off temperature = 0.0
    temperature=0.0,
    extra_body={
        "detailed_thinking": "off"
    }
)

print(response.choices[0].message.content)

**Professional Summary: Behavioral Health Insights from Feature Data**

This analysis provides a high-level interpretation of behavioral health patterns inferred from the provided wearable and survey data, grouped by domain:

* **Circadian Rhythm Stability**: Noted deviations suggest potential impacts on overall health, with fluctuations in step counts (steps_48phi, steps_8phi, steps_12p_reject, steps_IV) indicating possible circadian rhythm disruptions.
* **Cardiovascular Variability**: Elevated Heart Rate Variability (HRVD), particularly hrmax_std and hrentropy_std, may reflect increased cardiovascular stress levels, with peak heart rates (hrmax_mean, hrmax_max) suggesting high physical demand during work shifts.
* **Work/Shift Regularity**: Low median shift hours and specific entropy values (shift_type_shannon_entropy, shift_hours_shannon_entropy) imply irregular work schedules, potentially contributing to health disparities.
* **Daytime Inactivity**: Elevated non-step duration entr

In [ ]:
summary_text = response.choices[0].message.content

In [ ]:
classification_response = client.chat.completions.create(
    model="Llama-Nemotron-Nano-4B",
    messages=[
        {"role": "user", "content": build_classification_prompt(summary_text)}
    ],
    temperature=0.0,  # deterministic classification
    extra_body={
        "detailed_thinking": "off"
    }
)

burnout_label = classification_response.choices[0].message.content.strip()

print("Burnout Classification:", burnout_label)

Burnout Classification: High


**Whole Dataset**

In [ ]:
from openai import APIConnectionError
import time
import json

print(f"Processing {len(data.columns)} participants...")

summary_results = []

for participant_id, grouped_data in data.items():
    try:
        # Generate summary
        summary_response = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B",
            messages=[
                {"role": "user", "content": build_prompt(grouped_data)}
            ],
            temperature=0.3,
            extra_body={"detailed_thinking": "off"}
        )

        summary_text = summary_response.choices[0].message.content.strip()

        summary_results.append({
            "ID": participant_id,
            "llm_summary": summary_text
        })
        print(f"Processed {participant_id}")

    except APIConnectionError:
        print("CRITICAL ERROR: Could not connect to vLLM server. It may have crashed.")
        print("Please check the 'vllm.log' or restart the server cell.")
        break
    except Exception as e:
        print(f"Error processing {participant_id}: {e}")
        continue

with open("llm_summaries.json", "w") as f:
    json.dump(summary_results, f, indent=4)

print("saved summaries")

**Squential Processing**

In [ ]:
# Classification
with open("llm_summaries.json", "r") as f:
    summary_results = json.load(f)

results = []
for item in summary_results:
    participant_id = item["ID"]
    summary_text = item["llm_summary"]

    try:
        classification_response = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B",
            messages=[
                {"role": "user", "content": build_classification_prompt(summary_text)}
            ],
            temperature=0.0,
            max_tokens=3,
            extra_body={"detailed_thinking": "off"}
        )

        burnout_label = classification_response.choices[0].message.content.strip()

        results.append({
            "ID": participant_id,
            "llm_summary": summary_text,
            "llm_burnout_label": burnout_label
        })

        print(f"Classified {participant_id}: {burnout_label}")

**Batch Processing**

In [60]:
import concurrent.futures

classification_results = []

print(f"Running concurrent classification for {len(summary_results)} participants...")

def classify_participant(item):
    try:
        response = client.chat.completions.create(
            model="Llama-Nemotron-Nano-4B",
            messages=[
                {"role": "user", "content": build_classification_prompt(item["llm_summary"])}
            ],
            temperature=0.0,
            max_tokens=3,
            extra_body={"detailed_thinking": "off"}
        )
        burnout_label = response.choices[0].message.content.strip()
        return {
            "ID": item["ID"],
            "llm_summary": item["llm_summary"],
            "llm_burnout_label": burnout_label
        }
    except Exception as e:
        print(f"Error processing {item['ID']}: {e}")
        return None

# Adjust max_workers based on GPU capacity, I use L4 GPU.
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(classify_participant, item) for item in summary_results]

    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        result = future.result()
        if result:
            classification_results.append(result)

        if (i + 1) % 5 == 0:
            print(f"Completed {i+1}/{len(summary_results)}...")

print(f"Finished. Total classified: {len(classification_results)}")

Running concurrent classification for 44 participants...
Completed 5/44...
Completed 10/44...
Completed 15/44...
Completed 20/44...
Completed 25/44...
Completed 30/44...
Completed 35/44...
Completed 40/44...
Finished. Total classified: 44


In [61]:
df_results = pd.DataFrame(classification_results)

if not df_results.empty:
    print(df_results[["ID", "llm_burnout_label"]].head())
else:
    print("No results generated due to connection error.")

     ID llm_burnout_label
0  1102              High
1  1103              High
2  1115              High
3  1150              High
4  1108              High


In [23]:
df_results.to_csv("llm_predictions.csv", index=False)

In [62]:
ground_truth = pd.read_csv("/content/jbs_scores_post.csv")
df_results["ID"] = df_results["ID"].astype(str)
ground_truth.rename(columns={"u_id": "ID"}, inplace=True)
ground_truth["ID"] = ground_truth["ID"].astype(str)

# Merge
df_eval = df_results.merge(
    ground_truth,
    on="ID",
    how="inner"
)

print("Merged shape:", df_eval.shape)

Merged shape: (44, 10)


In [63]:
# Clean up LLM predictions (remove Markdown bolding like **High**)
df_eval["llm_burnout_label"] = (
    df_eval["llm_burnout_label"]
    .astype(str)
    .str.replace(r"[*]", "", regex=True) # Remove asterisks
    .str.strip()
    .str.capitalize()
)

if pd.api.types.is_numeric_dtype(df_eval["risk"]):
    df_eval["risk"] = df_eval["risk"].map({1: "High", 0: "Low"})


In [64]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(
    df_eval["risk"],
    df_eval["llm_burnout_label"]
)

print("Accuracy:", accuracy)

print("\nDetailed Report:")
print(classification_report(
    df_eval["risk"],
    df_eval["llm_burnout_label"]
))

Accuracy: 0.8409090909090909

Detailed Report:
              precision    recall  f1-score   support

        High       0.92      0.89      0.91        38
         Low       0.43      0.50      0.46         6

    accuracy                           0.84        44
   macro avg       0.67      0.70      0.68        44
weighted avg       0.85      0.84      0.85        44

